# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.83,78,100,15.49,TF,1698693503
1,1,kuanda,56.3161,116.0806,-10.78,51,12,0.21,RU,1698693503
2,2,tura,25.5198,90.2201,20.18,87,100,1.52,IN,1698693504
3,3,edinburgh of the seven seas,-37.0676,-12.3116,12.09,90,100,14.09,SH,1698693504
4,4,stanley,54.8680,-1.6985,6.77,93,75,3.09,GB,1698693504


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points('Lng','Lat',
                           geo=True,tiles='OSM', 
                           frame_width=700,frame_height=500,
                           color='City',size='Humidity',alpha=.75)

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Temperature between 21 and 27 degrees, Wind Speed less than 4.5, Cloudiness equal to 0
ideal_weather = city_data_df.loc[(city_data_df['Max Temp']>21)&
                                 (city_data_df['Max Temp']<27)&
                                (city_data_df['Wind Speed']<4.5)&
                                (city_data_df['Cloudiness']<10)]

# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,sassandra,4.9500,-6.0833,26.09,85,0,3.05,CI,1698693513
53,53,lompoc,34.6391,-120.4579,26.50,16,0,0.89,US,1698693517
100,100,guerrero negro,27.9769,-114.0611,25.26,32,0,2.54,MX,1698693528
105,105,dahanu,19.9667,72.7333,25.68,54,1,2.01,IN,1698693529
142,142,sur,22.5667,59.5289,26.48,66,7,2.38,OM,1698693536
184,184,ajdabiya,30.7554,20.2263,22.34,69,0,3.99,LY,1698693545
195,195,sanctuary point,-35.1114,150.6290,22.64,51,5,2.67,AU,1698693549
226,226,sitia,35.2000,26.0986,22.77,83,5,3.00,GR,1698693557
278,278,heroica caborca,30.7167,-112.1500,26.23,13,0,2.66,MX,1698693570
332,332,pacific grove,36.6177,-121.9166,23.83,22,0,2.06,US,1698693587


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
38,sassandra,CI,4.9500,-6.0833,85,
53,lompoc,US,34.6391,-120.4579,16,
100,guerrero negro,MX,27.9769,-114.0611,32,
105,dahanu,IN,19.9667,72.7333,54,
142,sur,OM,22.5667,59.5289,66,
184,ajdabiya,LY,30.7554,20.2263,69,
195,sanctuary point,AU,-35.1114,150.6290,51,
226,sitia,GR,35.2000,26.0986,83,
278,heroica caborca,MX,30.7167,-112.1500,13,
332,pacific grove,US,36.6177,-121.9166,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sassandra - nearest hotel: Hotel Le Pollet
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
guerrero negro - nearest hotel: Plaza sal paraiso
dahanu - nearest hotel: Crazy Crab
sur - nearest hotel: Sur Hotel
ajdabiya - nearest hotel: فندق المدينة
sanctuary point - nearest hotel: Dungowan Accommodation
sitia - nearest hotel: Itanos Hotel
heroica caborca - nearest hotel: Hotel Posada del Desierto
pacific grove - nearest hotel: Pacific Grove Inn
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
kismayo - nearest hotel: Kismayo Hotel
paithan - nearest hotel: No hotel found
ndele - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
bondoukou - nearest hotel: Hotel Akakro


,City,Country,Lat,Lng,Humidity,Hotel Name
38,sassandra,CI,4.9500,-6.0833,85,Hotel Le Pollet
53,lompoc,US,34.6391,-120.4579,16,Embassy Suites by Hilton Lompoc Central Coast
100,guerrero negro,MX,27.9769,-114.0611,32,Plaza sal paraiso
105,dahanu,IN,19.9667,72.7333,54,Crazy Crab
142,sur,OM,22.5667,59.5289,66,Sur Hotel
184,ajdabiya,LY,30.7554,20.2263,69,فندق المدينة
195,sanctuary point,AU,-35.1114,150.6290,51,Dungowan Accommodation
226,sitia,GR,35.2000,26.0986,83,Itanos Hotel
278,heroica caborca,MX,30.7167,-112.1500,13,Hotel Posada del Desierto
332,pacific grove,US,36.6177,-121.9166,22,Pacific Grove Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points('Lng','Lat',
                           geo=True,tiles='OSM', 
                           frame_width=700,frame_height=500,
                           color='City',size='Humidity',alpha=.75,hover_cols=['Hotel Name','Country'])

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)